# new draft

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OneHotEncoder
from zipfile import ZipFile
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from pathlib import Path
from geopy.distance import geodesic


user_rating_data = pd.read_csv('https://raw.githubusercontent.com/WayGo12/WayGoApp/main/assets/dataset/user_rating_dataset.csv')
places_data = pd.read_csv('https://raw.githubusercontent.com/WayGo12/WayGoApp/main/assets/dataset/places_dataset.csv')
user_data = pd.read_csv('https://raw.githubusercontent.com/WayGo12/WayGoApp/main/assets/dataset/user_id_dataset.csv')

data_recommend = pd.merge(user_rating_data.groupby('Place_ID')['Place_Rating'].mean(), places_data, on='Place_ID')
data_copy1 = data_recommend.copy()

def prepare_data(user_rating_data):
    data_collaborative_filtering = user_rating_data.copy()

    global user_to_user_encoded, user_encoded_to_user, resto_to_resto_encoded, resto_encoded_to_resto

    user_ids = data_collaborative_filtering['ID_user'].unique().tolist()
    user_to_user_encoded = {x: i for i, x in enumerate(user_ids)}
    user_encoded_to_user = {i: x for i, x in enumerate(user_ids)}

    resto_ids = data_collaborative_filtering['Place_ID'].unique().tolist()
    resto_to_resto_encoded = {x: i for i, x in enumerate(resto_ids)}
    resto_encoded_to_resto = {i: x for i, x in enumerate(resto_ids)}


    data_collaborative_filtering['user'] = data_collaborative_filtering['ID_user'].map(user_to_user_encoded)
    data_collaborative_filtering['place'] = data_collaborative_filtering['Place_ID'].map(resto_to_resto_encoded)

    num_users = len(user_to_user_encoded)
    num_resto = len(resto_encoded_to_resto)
    data_collaborative_filtering['Place_Rating'] = data_collaborative_filtering['Place_Rating'].values.astype(np.float32)
    min_rating = min(data_collaborative_filtering['Place_Rating'])
    max_rating = max(data_collaborative_filtering['Place_Rating'])

    data_collaborative_filtering = data_collaborative_filtering.sample(frac=1, random_state=42)

    x = data_collaborative_filtering[['user', 'place']].values
    y = data_collaborative_filtering['Place_Rating'].apply(lambda x: (x - min_rating) / (max_rating - min_rating)).values
    train_indices = int(0.8 * data_collaborative_filtering.shape[0])
    x_train, x_val, y_train, y_val = (
        x[:train_indices],
        x[train_indices:],
        y[:train_indices],
        y[train_indices:]
    )

    return x_train, x_val, y_train, y_val, num_users, num_resto, user_to_user_encoded, user_encoded_to_user, resto_to_resto_encoded, resto_encoded_to_resto


def create_model(num_users, num_resto, embedding_size):
    @keras.saving.register_keras_serializable()
    class RecommenderNet(tf.keras.Model):
        def __init__(self, num_users, num_resto, embedding_size, **kwargs):
            super(RecommenderNet, self).__init__(**kwargs)
            self.num_users = num_users
            self.num_resto = num_resto
            self.embedding_size = embedding_size
            self.user_embedding = layers.Embedding(
                num_users,
                embedding_size,
                embeddings_initializer='he_normal',
                embeddings_regularizer=keras.regularizers.l2(1e-6)
            )
            self.user_bias = layers.Embedding(num_users, 1)
            self.resto_embedding = layers.Embedding(
                num_resto,
                embedding_size,
                embeddings_initializer='he_normal',
                embeddings_regularizer=keras.regularizers.l2(1e-6)
            )
            self.resto_bias = layers.Embedding(num_resto, 1)

        def call(self, inputs):
            user_vector = self.user_embedding(inputs[:, 0])
            user_bias = self.user_bias(inputs[:, 0])
            resto_vector = self.resto_embedding(inputs[:, 1])
            resto_bias = self.resto_bias(inputs[:, 1])

            dot_user_resto = tf.tensordot(user_vector, resto_vector, 2)

            x = dot_user_resto + user_bias + resto_bias
            return tf.nn.sigmoid(x)

    model = RecommenderNet(num_users, num_resto, embedding_size)
    model.compile(
        loss=tf.keras.losses.BinaryCrossentropy(),
        optimizer=keras.optimizers.Adam(learning_rate=0.001),
        metrics=[tf.keras.metrics.RootMeanSquaredError()]
    )

    return model


def train_model(model, x_train, y_train, x_val, y_val, batch_size, epochs):
    history = model.fit(
        x=x_train,
        y=y_train,
        batch_size=batch_size,
        epochs=epochs,
        validation_data=(x_val, y_val)
    )
    return history


def collaborative_filtering_recommender(places_data, user_rating_data, embedding_size, batch_size, epochs, save_model_path=None):
    x_train, x_val, y_train, y_val, num_users, num_resto, user_to_user_encoded, user_encoded_to_user, resto_to_resto_encoded, resto_encoded_to_resto = prepare_data(user_rating_data)
    model = create_model(num_users, num_resto, embedding_size)
    train_model(model, x_train, y_train, x_val, y_val, batch_size, epochs)
    if save_model_path:
        model.save(save_model_path, save_format='tf')
        converter = tf.lite.TFLiteConverter.from_saved_model(save_model_path)
        tflite_model = converter.convert()
        tflite_model_path = save_model_path + '.tflite'
        with open(tflite_model_path, 'wb') as f:
            f.write(tflite_model)

    return model


def recommend_by_collaborative_filtering(model, places_data, user_rating_data, user_id):
    resto_df = places_data
    df = user_rating_data
    resto_visited_by_user = df[df.ID_user == user_id]
    resto_not_visited = resto_df[~resto_df['Place_ID'].isin(resto_visited_by_user.Place_ID.values)]['Place_ID']
    resto_not_visited = list(
        set(resto_not_visited).intersection(resto_to_resto_encoded.keys())
    )
    resto_not_visited = list(map(resto_to_resto_encoded.get, resto_not_visited))

    user_encoder = user_to_user_encoded[user_id]
    user_resto_array = np.hstack(([[user_encoder]] * len(resto_not_visited), np.array(resto_not_visited)[:, None]))
    ratings = model.predict(user_resto_array).flatten()
    top_ratings_indices = ratings.argsort()[-50:][::-1]
    recommended_resto_ids = [resto_encoded_to_resto.get(resto_not_visited[index]) for index in top_ratings_indices]
    recommended_resto_info = resto_df[resto_df['Place_ID'].isin(recommended_resto_ids)][['Place_Name', 'Latitude', 'Longitude', 'Place_Category']].values.tolist()

    recommended_places = {
        'Nama Tempat': [],
        'Latitude': [],
        'Longitude': [],
        'Kategori' : []
    }

    for place_info in recommended_resto_info:
        recommended_places['Nama Tempat'].append(place_info[0])
        recommended_places['Latitude'].append(place_info[1])
        recommended_places['Longitude'].append(place_info[2])
        recommended_places['Kategori'].append(place_info[3])

    return recommended_places

In [3]:
places_data["Place_Region"].unique()

array([' East Bali', ' North Bali', ' South Bali', ' West Bali'],
      dtype=object)

In [27]:
from geopy.distance import geodesic

def haversine_distance(coord1, coord2):
    # Calculate the distance using the Haversine formula
    return geodesic(coord1, coord2).kilometers

def find_closest_place(current_place, places_data, valid_categories):
    current_coord = (places_data['Latitude'][current_place], places_data['Longitude'][current_place])
    valid_places = [(i, (places_data['Latitude'][i], places_data['Longitude'][i])) for i in range(len(places_data['Nama Tempat']))
                    if places_data['Kategori'][i] in valid_categories and i != current_place]
    if not valid_places:
        return None
    closest_place, closest_distance = min(valid_places, key=lambda x: haversine_distance(current_coord, x[1]))
    return closest_place

def delete_processed_place(places_data, index):
    del places_data['Nama Tempat'][index]
    del places_data['Latitude'][index]
    del places_data['Longitude'][index]
    del places_data['Kategori'][index]

def generate_rundown_for_user(user_data):
    places_data = {
        'Nama Tempat': user_data['Nama Tempat'],
        'Latitude': user_data['Latitude'],
        'Longitude': user_data['Longitude'],
        'Kategori': user_data['Kategori']
    }

    # Find the first accommodation place
    accommodation_place = user_data['Kategori'].index('Accomodation')

    # 9 PM - 6 AM: Accommodation
    rundown = [{'Jam Rundown': '9 PM - 6 AM',
                'Nama Tempat': places_data['Nama Tempat'][accommodation_place],
                'Kategori': places_data['Kategori'][accommodation_place],
                'Jarak Tempat': None}]

    # 6 AM - 8 AM: Culinary (Breakfast)
    culinary_place = find_closest_place(accommodation_place, places_data, ['Culinary'])
    distance = haversine_distance((places_data['Latitude'][accommodation_place], places_data['Longitude'][accommodation_place]),
                                  (places_data['Latitude'][culinary_place], places_data['Longitude'][culinary_place]))
    rundown.append({'Jam Rundown': '6 AM - 8 AM',
                    'Nama Tempat': places_data['Nama Tempat'][culinary_place],
                    'Kategori': places_data['Kategori'][culinary_place],
                    'Jarak Tempat': distance})
    delete_processed_place(places_data, accommodation_place)

    # 8 AM - 10 AM: Activity
    valid_activity_categories = ['Nautical', 'Nature', 'History', 'Entertainment', 'Shopping', 'Art']
    activity_place = find_closest_place(culinary_place, places_data, valid_activity_categories)
    distance = haversine_distance((places_data['Latitude'][culinary_place], places_data['Longitude'][culinary_place]),
                                  (places_data['Latitude'][activity_place], places_data['Longitude'][activity_place]))
    rundown.append({'Jam Rundown': '8 AM - 10 AM',
                    'Nama Tempat': places_data['Nama Tempat'][activity_place],
                    'Kategori': places_data['Kategori'][activity_place],
                    'Jarak Tempat': distance})
    delete_processed_place(places_data, culinary_place)

    # 10 AM - 12 PM: Activity
    valid_activity_categories_10_to_12 = ['Nautical', 'Nature', 'History', 'Entertainment', 'Shopping', 'Art']
    activity_place_10_to_12 = find_closest_place(activity_place, places_data, valid_activity_categories_10_to_12)
    distance = haversine_distance((places_data['Latitude'][activity_place], places_data['Longitude'][activity_place]),
                                  (places_data['Latitude'][activity_place_10_to_12], places_data['Longitude'][activity_place_10_to_12]))
    rundown.append({'Jam Rundown': '10 AM - 12 PM',
                    'Nama Tempat': places_data['Nama Tempat'][activity_place_10_to_12],
                    'Kategori': places_data['Kategori'][activity_place_10_to_12],
                    'Jarak Tempat': distance})
    delete_processed_place(places_data, activity_place)

    # 12 PM - 1 PM: Culinary (Lunch)
    culinary_place_12_to_1 = find_closest_place(activity_place_10_to_12, places_data, ['Culinary'])
    distance = haversine_distance((places_data['Latitude'][activity_place_10_to_12], places_data['Longitude'][activity_place_10_to_12]),
                                  (places_data['Latitude'][culinary_place_12_to_1], places_data['Longitude'][culinary_place_12_to_1]))
    rundown.append({'Jam Rundown': '12 PM - 1 PM',
                    'Nama Tempat': places_data['Nama Tempat'][culinary_place_12_to_1],
                    'Kategori': places_data['Kategori'][culinary_place_12_to_1],
                    'Jarak Tempat': distance})
    delete_processed_place(places_data, activity_place_10_to_12)

    # 1 PM - 3 PM: Activity
    valid_activity_categories_1_to_3 = ['Nautical', 'Nature', 'History', 'Entertainment', 'Shopping', 'Art']
    activity_place_1_to_3 = find_closest_place(culinary_place_12_to_1, places_data, valid_activity_categories_1_to_3)
    distance = haversine_distance((places_data['Latitude'][culinary_place_12_to_1], places_data['Longitude'][culinary_place_12_to_1]),
                                  (places_data['Latitude'][activity_place_1_to_3], places_data['Longitude'][activity_place_1_to_3]))
    rundown.append({'Jam Rundown': '1 PM - 3 PM',
                    'Nama Tempat': places_data['Nama Tempat'][activity_place_1_to_3],
                    'Kategori': places_data['Kategori'][activity_place_1_to_3],
                    'Jarak Tempat': distance})
    delete_processed_place(places_data, culinary_place_12_to_1)

    # 3 PM - 5 PM: Activity
    valid_activity_categories_3_to_5 = ['Nautical', 'Nature', 'History', 'Entertainment', 'Shopping', 'Art']
    activity_place_3_to_5 = find_closest_place(activity_place_1_to_3, places_data, valid_activity_categories_3_to_5)
    distance = haversine_distance((places_data['Latitude'][activity_place_1_to_3], places_data['Longitude'][activity_place_1_to_3]),
                                  (places_data['Latitude'][activity_place_3_to_5], places_data['Longitude'][activity_place_3_to_5]))
    rundown.append({'Jam Rundown': '3 PM - 5 PM',
                    'Nama Tempat': places_data['Nama Tempat'][activity_place_3_to_5],
                    'Kategori': places_data['Kategori'][activity_place_3_to_5],
                    'Jarak Tempat': distance})
    delete_processed_place(places_data, activity_place_1_to_3)

    # 5 PM - 7 PM: Culinary (Dinner)
    culinary_place_5_to_7 = find_closest_place(activity_place_3_to_5, places_data, ['Culinary'])
    distance = haversine_distance((places_data['Latitude'][activity_place_3_to_5], places_data['Longitude'][activity_place_3_to_5]),
                                  (places_data['Latitude'][culinary_place_5_to_7], places_data['Longitude'][culinary_place_5_to_7]))
    rundown.append({'Jam Rundown': '5 PM - 7 PM',
                    'Nama Tempat': places_data['Nama Tempat'][culinary_place_5_to_7],
                    'Kategori': places_data['Kategori'][culinary_place_5_to_7],
                    'Jarak Tempat': distance})
    delete_processed_place(places_data, activity_place_3_to_5)

    # 7 PM - 9 PM: Shopping
    shopping_place_7_to_9 = find_closest_place(culinary_place_5_to_7, places_data, ['Shopping'])
    distance = haversine_distance((places_data['Latitude'][culinary_place_5_to_7], places_data['Longitude'][culinary_place_5_to_7]),
                                  (places_data['Latitude'][shopping_place_7_to_9], places_data['Longitude'][shopping_place_7_to_9]))
    rundown.append({'Jam Rundown': '7 PM - 9 PM',
                    'Nama Tempat': places_data['Nama Tempat'][shopping_place_7_to_9],
                    'Kategori': places_data['Kategori'][shopping_place_7_to_9],
                    'Jarak Tempat': distance})
    delete_processed_place(places_data, culinary_place_5_to_7)

    return rundown

In [5]:
save_model_path = 'model'  # Replace with the desired save path
recommender = collaborative_filtering_recommender(places_data, user_rating_data, embedding_size=50, batch_size=8, epochs=10, save_model_path=save_model_path)

Epoch 1/10
100/100 [==============================] - 2s 6ms/step - loss: 0.6938 - root_mean_squared_error: 0.1920 - val_loss: 0.6917 - val_root_mean_squared_error: 0.1894
Epoch 2/10
100/100 [==============================] - 0s 4ms/step - loss: 0.6913 - root_mean_squared_error: 0.1887 - val_loss: 0.6902 - val_root_mean_squared_error: 0.1874
Epoch 3/10
100/100 [==============================] - 0s 4ms/step - loss: 0.6870 - root_mean_squared_error: 0.1830 - val_loss: 0.6891 - val_root_mean_squared_error: 0.1859
Epoch 4/10
100/100 [==============================] - 0s 4ms/step - loss: 0.6824 - root_mean_squared_error: 0.1767 - val_loss: 0.6873 - val_root_mean_squared_error: 0.1835
Epoch 5/10
100/100 [==============================] - 0s 4ms/step - loss: 0.6812 - root_mean_squared_error: 0.1749 - val_loss: 0.6859 - val_root_mean_squared_error: 0.1815
Epoch 6/10
100/100 [==============================] - 0s 4ms/step - loss: 0.6763 - root_mean_squared_error: 0.1679 - val_loss: 0.6847 - val_

In [6]:
save_model_path = 'model'  # Replace with the desired save path
recommender.save(save_model_path, save_format='tf')

In [7]:
save_model_path = 'model_weights.h5'  # Replace with the desired save path
recommender.save_weights(save_model_path)

In [8]:
import pickle

# Save the model as a pickle file
with open('model.pkl', 'wb') as f:
    pickle.dump(recommender, f)

In [9]:
user_id = 133 # Replace 123 with the actual user ID
recommendations = recommend_by_collaborative_filtering(recommender, places_data, user_rating_data, user_id)

9/9 [==============================] - 0s 2ms/step


In [10]:
recommendations

{'Nama Tempat': ['Warung Babi Guling Pak Malen',
  'Ibu Oka Babi Guling',
  'Locavore',
  "Naughty Nuri's Warung",
  'Swept Away at The Samaya',
  'Melting Wok Warung',
  'The Sayan House',
  'Bridges Bali',
  'Kubu at Mandapa',
  'Bridges Bali',
  'Uma Cucina',
  'The Sayan House',
  'Swept Away at The Samaya',
  'Melting Wok Warung',
  'Alchemy',
  'Spice Beach Club',
  'Spice Beach Club',
  'Ku De Ta',
  'The Bistrot',
  'Revolver Espresso',
  'Mozzarella Restaurant and Bar',
  'Kunti III',
  'Corner House Bali',
  'Warung Eropa',
  "Warung D'Sawah",
  'The Menjangan Resort Bali',
  'Warung Banyuwedang',
  'Bali Tower Restaurant',
  'Damai Restaurant',
  'Menjangan Dynasty Resort - Pasir Putih Restaurant',
  'Buda Bakery & Resto',
  'WakaGangga Restaurant',
  'Yeh Panes Restaurant',
  'Soka Indah Restaurant & Cafe',
  'Menega Cafe',
  'Cuca Flavor',
  'Blue Marlin Bali',
  'Bluefin Bali',
  'Lesehan Segara',
  'Babi Guling Pak Malen',
  "Warung D'Sawah",
  'Palasari Dam Terrace',
  

In [11]:
df_rec = pd.DataFrame(recommendations)
df_rec

,Nama Tempat,Latitude,Longitude,Kategori
0,Warung Babi Guling Pak Malen,-8.684800,115.169100,Culinary
1,Ibu Oka Babi Guling,-8.505770,115.264300,Culinary
2,Locavore,-8.509960,115.263500,Culinary
3,Naughty Nuri's Warung,-8.492310,115.253300,Culinary
4,Swept Away at The Samaya,-8.513630,115.237200,Culinary
5,Melting Wok Warung,-8.509530,115.264200,Culinary
6,The Sayan House,-8.498610,115.243800,Culinary
7,Bridges Bali,-8.505070,115.254200,Culinary
8,Kubu at Mandapa,-8.485760,115.244300,Culinary
9,Bridges Bali,-8.505070,115.254200,Culinary


In [12]:
rundown1 = generate_rundown_for_user(recommendations)
rundown1

NameError: name 'generate_rundown_for_user' is not defined

# Start Point

**East Bali:**
*   Latitude: Approximately -8.471148
*   Longitude: Approximately 115.665717

**North Bali:**
*   Latitude: Approximately -8.182740
*   Longitude: Approximately 115.136624

**West Bali:**
*   Latitude: Approximately -8.409518
*   Longitude: Approximately 114.979448

**South Bali:**
*   Latitude: Approximately -8.409518
*   Longitude: Approximately 115.188916